In [1]:
import numpy as np
import matplotlib.pyplot as plt


def integ(xs,ys):
    Y = []
    X = []
    temp_y = 0.0
    temp_x = 0.0
    for i in range(len(xs)-1):
        temp_x = (xs[i+1]+xs[i])/2
        X.append(temp_x)

        temp_y += (xs[i+1]-xs[i])*(ys[i+1]+ys[i])/2
        Y.append(temp_y)
    return np.array(X),np.array(Y)

def diff(x,y):
    h = x[1]-x[0]
    y0 = (-3*y[0]+4*y[1]-y[2])/(2*h)
    y1 = (y[2:]-y[:-2])/(2*h)
    y2 = (y[-3]-4*y[-2]+3*y[-1])/(2*h)
    return np.concatenate(([y0],y1,[y2]))

def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

"""def hankel_matrix(array, l):

    HM = np.zeros([series.shape[0] - l, l])

    for i in range(0, series.shape[0] - l):
        HM[i] = np.squeeze(series[i:i + l, 0])
    return HM"
"""

def hankel_matrix(array, l, each = 1):
    array = array[::each]
    HM = np.zeros([len(array) - l, l])

    for i in range(0, len(array) - l):
        HM[i] = np.squeeze(array[i:i + l])
    return HM

def eigenvalues_plot(__x,n,each = 1,num_l = None, MinMax = False):
    HM = hankel_matrix(__x, n,each)

    S = np.linalg.svd(HM,compute_uv=False)
    if MinMax:
        S /= max(S)

    plt.figure(figsize=(11, 6))

    plt.plot(S[:num_l]**.5,'o')
    plt.plot(S[:num_l]**.5,'--')

    #plt.xlim(-1,20)


    plt.xlabel('N', size=23)
    plt.ylabel('λ', size=23)

    plt.tick_params(axis='both', which='major', labelsize=25,length=8, width=4)
    plt.grid(which='major',
        color = 'gray',
        linewidth = 0.8)
    plt.minorticks_on()
    plt.grid(which='minor',
        color = 'gray',
        linestyle = '--',
        linewidth = 0.5)
    plt.legend(shadow=True, ncol=1, fontsize=15)
    plt.show()


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




class SSA(object):

    __supported_types = (pd.Series, np.ndarray, list)

    def __init__(self, tseries, L, save_mem=True):
        """
        Decomposes the given time series with a singular-spectrum analysis. Assumes the values of the time series are
        recorded at equal intervals.
        
        Parameters
        ----------
        tseries : The original time series, in the form of a Pandas Series, NumPy array or list.
        L : The window length. Must be an integer 2 <= L <= N/2, where N is the length of the time series.
        save_mem : Conserve memory by not retaining the elementary matrices. Recommended for long time series with
            thousands of values. Defaults to True.
        
        Note: Even if an NumPy array or list is used for the initial time series, all time series returned will be
        in the form of a Pandas Series or DataFrame object.
        """
        
        # Tedious type-checking for the initial time series
        if not isinstance(tseries, self.__supported_types):
            raise TypeError("Unsupported time series object. Try Pandas Series, NumPy array or list.")
        
        # Checks to save us from ourselves
        self.N = len(tseries)
        if not 2 <= L <= self.N/2:
            raise ValueError("The window length must be in the interval [2, N/2].")
        
        self.L = L
        self.orig_TS = pd.Series(tseries)
        self.K = self.N - self.L + 1
        
        # Embed the time series in a trajectory matrix
        self.X = np.array([self.orig_TS.values[i:L+i] for i in range(0, self.K)]).T
        
        # Decompose the trajectory matrix
        self.U, self.Sigma, VT = np.linalg.svd(self.X)
        self.d = np.linalg.matrix_rank(self.X)
        
        self.TS_comps = np.zeros((self.N, self.d))
        
        if not save_mem:
            # Construct and save all the elementary matrices
            self.X_elem = np.array([ self.Sigma[i]*np.outer(self.U[:,i], VT[i,:]) for i in range(self.d) ])

            # Diagonally average the elementary matrices, store them as columns in array.
            for i in range(self.d):
                X_rev = self.X_elem[i, ::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.V = VT.T
        else:
            # Reconstruct the elementary matrices without storing them
            for i in range(self.d):
                X_elem = self.Sigma[i]*np.outer(self.U[:,i], VT[i,:])
                X_rev = X_elem[::-1]
                self.TS_comps[:,i] = [X_rev.diagonal(j).mean() for j in range(-X_rev.shape[0]+1, X_rev.shape[1])]
            
            self.X_elem = "Re-run with save_mem=False to retain the elementary matrices."
            
            # The V array may also be very large under these circumstances, so we won't keep it.
            self.V = "Re-run with save_mem=False to retain the V matrix."
        
        # Calculate the w-correlation matrix.
        self.calc_wcorr()
            
    def components_to_df(self, n=0):
        """
        Returns all the time series components in a single Pandas DataFrame object.
        """
        if n > 0:
            n = min(n, self.d)
        else:
            n = self.d
        
        # Create list of columns - call them F0, F1, F2, ...
        cols = ["F{}".format(i) for i in range(n)]
        return pd.DataFrame(self.TS_comps[:, :n], columns=cols, index=self.orig_TS.index)
            

    def reconstruct(self, indices):
        """
        Reconstructs the time series from its elementary components, using the given indices. Returns a Pandas Series
        object with the reconstructed time series.
        
        Parameters
        ----------
        indices: An integer, list of integers or slice(n,m) object, representing the elementary components to sum.
        """
        if isinstance(indices, int): indices = [indices]
        
        ts_vals = self.TS_comps[:,indices].sum(axis=1)
        return pd.Series(ts_vals, index=self.orig_TS.index)

    def calc_wcorr(self):
        """
        Calculates the w-correlation matrix for the time series.
        """
             
        # Calculate the weights
        w = np.array(list(np.arange(self.L)+1) + [self.L]*(self.K-self.L-1) + list(np.arange(self.L)+1)[::-1])
        
        def w_inner(F_i, F_j):
            return w.dot(F_i*F_j)
        
        # Calculated weighted norms, ||F_i||_w, then invert.
        F_wnorms = np.array([w_inner(self.TS_comps[:,i], self.TS_comps[:,i]) for i in range(self.d)])
        F_wnorms = F_wnorms**-0.5
        
        # Calculate Wcorr.
        self.Wcorr = np.identity(self.d)
        for i in range(self.d):
            for j in range(i+1,self.d):
                self.Wcorr[i,j] = abs(w_inner(self.TS_comps[:,i], self.TS_comps[:,j]) * F_wnorms[i] * F_wnorms[j])
                self.Wcorr[j,i] = self.Wcorr[i,j]

    def plot_wcorr(self, min=None, max=None):
        """
        Plots the w-correlation matrix for the decomposed time series.
        """
        if min is None:
            min = 0
        if max is None:
            max = self.d
        
        if self.Wcorr is None:
            self.calc_wcorr()
        
        ax = plt.imshow(self.Wcorr)
        plt.xlabel(r"$\tilde{F}_i$")
        plt.ylabel(r"$\tilde{F}_j$")
        plt.colorbar(ax.colorbar, fraction=0.045)
        ax.colorbar.set_label("$W_{i,j}$")
        plt.clim(0,1)
        
        # For plotting purposes:
        if max == self.d:
            max_rnge = self.d-1
        else:
            max_rnge = max
        
        plt.xlim(min-0.5, max_rnge+0.5)
        plt.ylim(max_rnge+0.5, min-0.5)


In [3]:
'''
Visualization tools for MobileSensorData project
'''

import os
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

import plotly.graph_objects as go
import plotly.express as px

sensor_names = {
    'accm': 'Accelerometer',
    'gyrm': 'Gyroscope',
    'magm': 'Magnetometer',
    'grvm': 'Gravity Sensor',
    'lacm': 'Linear acceleration',
    'rotm': 'Rotation sensor',
}

sensor_colors = {
    'accm': 'b',
    'gyrm': 'g',
    'magm': 'r',
    'grvm': 'c',
    'lacm': 'm',
    'rotm': 'y',
}


def get_games(path):
    '''Get list of all games in the directory

    Parameters:
    - `path`: path to the directory

    Output:
    - `games`: list of games
    '''

    files = os.listdir(path)
    games = list(set([file[:-8] for file in files]))
    return games


def plot_time_sampling_stats(path, games=None, delimiter=',', decimal='.'):
    '''Plot sample length distribution

    Parameters:
    - `path': path to a folder containing .csv data of games
    - `games`: list of games that would be combined

    Notes:
    This function combines all games' data (only `time` column)
    into one pd.Dataframe.
    '''

    if games is None:
        print('No games was entered. '
              'All data will be processed.')
        games = get_games(path)

    game_data = pd.DataFrame()

    for game in games:
        for sensor in sensor_names:
            filename = path + game + sensor + '.csv'

            data = pd.read_csv(filename, delimiter=delimiter, decimal=decimal)
            time = data['time'].values
            samples = np.zeros([time.shape[0] - 1])
            for i in range(time.shape[0] - 1):
                samples[i] = time[i + 1] - time[i]

            game_data = game_data.append(pd.DataFrame({'time_samples': samples,
                                                       'sensor': [sensor_names[sensor]] * (time.shape[0] - 1),
                                                       'game': [game] * (time.shape[0] - 1)}),
                                         ignore_index=True)

    fig = px.histogram(game_data, x="time_samples",
                       color="sensor",
                       histnorm='percent',
                       template='plotly_white',
                       barmode='overlay')

    fig.for_each_trace(
        lambda trace: trace.update(name=trace.name.replace("=", ": ")),
    )

    fig.layout.yaxis.title.text = 'Count'
    fig.layout.xaxis.title.text = 'Time samples'
    fig.layout.height = 500

    fig.show()


def plot_game(path, game, sensors, delimiter=',', decimal='.'):
    '''Plot game data

    Parameters:
    - `path': path to a folder containing .csv data of games
    - `game`: name of game to be plotted
    - `sensors`: list of sensors
    - `delimiter`: csv-file delimiter
    - `decimal`: csv-file decimal delimiter
    '''

    for sensor in sensors:
        filename = path + game + sensor + '.csv'
        data = pd.read_csv(filename, delimiter=delimiter, decimal=decimal)

        fig = go.Figure()
        fig.add_scatter(x=data['time'], y=data['X_value'], mode='lines', name='X_value')
        fig.add_scatter(x=data['time'], y=data['Y_value'], mode='lines', name='Y_value')
        fig.add_scatter(x=data['time'], y=data['Z_value'], mode='lines', name='Z_value')

        fig.for_each_trace(
            lambda trace: trace.update(name=trace.name.replace("_value", "")),
        )

        fig.layout.template = 'plotly_white'

        fig.show()


def plot_phase_track(track, color=None):
    '''Plot given phase trajctory

    Parameters:
    - `track`: 3D or 2D phase trajectory
    '''

    fig = go.Figure()

    if track.shape[-1] == 2:
        fig.add_scatter(x=track[:, 0], y=track[:, 1],
                        marker_color=color,
                        mode='lines',
                        name='Phase track')

        fig.add_trace(go.Scatter(x=[track[0, 0]],
                                 y=[track[0, 1]],
                                 mode='markers',
                                 marker_size=10,
                                 marker_color='rgba(255, 10, 0, .7)',
                                 name='Start point'))

        fig.add_trace(go.Scatter(x=[track[-1, 0]],
                                 y=[track[-1, 1]],
                                 mode='markers',
                                 marker_size=10,
                                 marker_color='rgba(10, 250, 250, .7)',
                                 name='End point'))

    elif track.shape[-1] == 3:
        fig.add_scatter3d(x=track[:, 0], y=track[:, 1], z=track[:, 2],
                          marker_color=color,
                          mode='lines',
                          name='Phase track')

        fig.add_trace(go.Scatter3d(x=[track[0, 0]],
                                   y=[track[0, 1]],
                                   z=[track[0, 2]],
                                   mode='markers',
                                   marker_size=10,
                                   marker_color='rgba(255, 10, 0, .7)',
                                   name='Start point'))

        fig.add_trace(go.Scatter3d(x=[track[-1, 0]],
                                   y=[track[-1, 1]],
                                   z=[track[-1, 2]],
                                   mode='markers',
                                   marker_size=10,
                                   marker_color='rgba(10, 250, 250, .7)',
                                   name='End point'))

    else:
        raise ValueError('Check dimensionality of phase track')
        return

    fig.layout.template = 'plotly_white'
    fig.show()


def phase_track(series, l, n_components, plot_correlation_matrix=False):
    '''Get phase trajectory projection of series.

    Parameters:
    - `series`: 2Darray of shape [duration, 1]
    - `l`: dimensionality of feature space.
    - `n_components`: Number of components to keep
    while applying PCA to resulting trajectory.

    Output:
    - projection: projection of phase trajectory
    on the principal components.
    - basis: principal axes in feature space.
    '''

    phase = to_phase_space(series, l)

    if plot_correlation_matrix:
        plot_correlation(phase)

    model = PCA(n_components=n_components)
    projection = model.fit_transform(phase)
    basis = model.components_
    print('Explained variation'
          ' for {} principal components: {}'.format(n_components,
                                                    model.explained_variance_ratio_))
    print('Cumulative explained variation'
          'for {} principal components: {}\n'.format(n_components,
                                                     np.sum(model.explained_variance_ratio_)))
    return projection, basis


def plot_correlation(phase_track):
    '''Plot correlation matrix

    Parameters:
    - `phase_track`: phase trajectory
    '''

    fig = go.Figure(data=go.Heatmap(z=np.corrcoef(phase_track.T), colorscale='Viridis'))
    fig.update_layout(width=700, height=700, yaxis = dict({'autorange': 'reversed'}))
    fig.show()

def to_phase_space(series, l):
    '''Get phase trajectory of series.

    Parameters:
    - `series`: 2Darray of shape [duration, 1]
    - `l`: dimensionality of feature space.

    Output:
    - `phase`: phase trajectory
    '''

    phase = np.zeros([series.shape[0] - l, l])

    for i in range(0, series.shape[0] - l):
        phase[i] = np.squeeze(series[i:i + l, 0])
    return phase

def Sphere_projection(track_init,n = 1):

    _x = track_init[:,0] - track_init[:,0].mean()
    _y = track_init[:,1] - track_init[:,1].mean()
    _z = track_init[:,2] - track_init[:,2].mean()

    track = np.array([_x,_y,_z]).T


    for i in range(len(track)):
        track[i] = track[i]/((track[i,:]**2).sum())**.5

    t, p = np.mgrid[0:np.pi:100j, 0:2*np.pi:100j]

    x_s = 0.98*np.sin(t)*np.cos(p)
    y_s = 0.98*np.sin(t)*np.sin(p)
    z_s = 0.98*np.cos(t)

    fig = go.Figure()

    fig.add_trace(go.Surface(x=x_s,
                               y=y_s,
                               z=z_s,
                               showscale=False,
                               surfacecolor = z_s))

    fig.add_trace(go.Scatter3d(x=track[:,0][::n],
                                 y=track[:,1][::n],
                                 z=track[:,2][::n],
                                 #mode='markers',
                                 marker=dict(
                                             size=2,
                                             line=dict(
                                                        width=0.1
                                                      )

                                             ),
                                 name='trajectory'
                                )
                    )
        
        
        
    fig.add_trace(go.Scatter3d(x=[track[0, 0]],
                               y=[track[0, 1]],
                               z=[track[0, 2]],
                               mode='markers',
                               marker_size=10,
                               marker_color='rgba(255, 10, 0, .7)',
                               name='Start point'))

    fig.add_trace(go.Scatter3d(x=[track[-1, 0]],
                               y=[track[-1, 1]],
                               z=[track[-1, 2]],
                               mode='markers',
                               marker_size=10,
                               marker_color='rgba(10, 250, 250, .7)',
                               name='End point'))



    fig.layout.template = 'plotly_white'

    return fig


In [12]:
import numpy as np
import pandas as pd
import math as m
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from scipy import *
import scipy.linalg
from scipy.linalg import eig, eigh
from sklearn.datasets import make_swiss_roll
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph, NearestNeighbors
from sklearn.decomposition import KernelPCA, PCA
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error

#import scaleogram as scg
import matplotlib.pyplot as plt
import warnings

from visuals import *
from my_lib import *
from SSA_lib import SSA

In [14]:
warnings.simplefilter('ignore')

plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.unicode'] = True
plt.rcParams['figure.figsize'] = 5, 5
plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 8

In [16]:
dt = 450*10
#data = pd.read_csv('data/long_walk_100_acc.csv', delimiter =';', decimal=',')[3815+6*455:3815+6*455+dt]
data = pd.read_csv('data/long_walk_100_acc.csv', delimiter =';', decimal=',')[7007:7007+dt]
#data = pd.read_csv('data/long_walk_100_acc.csv', delimiter =';', decimal=',')[3785:8000]
#data = pd.read_csv('data/home_lin_10_lac.csv', delimiter =';', decimal=',')[:]
frecuency = len(data)/(data['time'].values[-1]-data['time'].values[0])
assert 480 < frecuency < 520

x_acc = ( (data['X_value'].values)**2 + (data['Y_value'].values)**2 + (data['Z_value'].values)**2)**.5
_m = np.mean(x_acc)
#x_acc -= _m
t = (data['time'].values).astype(float).reshape([-1,])
t = np.linspace(0,t[-1]-t[0],len(x_acc))

fig = go.Figure()
fig.add_scatter(y = x_acc, mode='lines', name='Sum squares')
fig.show()

In [17]:
accel_ssa = SSA(x_acc, 500)

x_acc_clear = accel_ssa.reconstruct(slice(0,5))

def HankelMatrix(X, L):  
    N = X.shape[0]
    return scipy.linalg.hankel(X[ : N - L + 1], X[N - L : N])

def inverse_HankelMatrix(X, L):  
    N = X.shape[0]
    return scipy.linalg.hankel(X[ : N - L + 1], X[N - L : N])

In [10]:
fig = go.Figure()

fig.add_scatter(y = x_acc,
                mode='lines',
                name='Full')

fig.add_scatter(y = x_acc_clear,
                mode='lines',
                name='0-5')

fig.show()

In [13]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [ ]:
plt.plot(t[1000:1600]-t[1000], x_acc[1000:1600],'--')
plt.plot(t[1000:1600]-t[1000], x_acc_clear[1000:1600])
plt.ylabel('x(t)', size=16)
plt.xlabel('t,c', size=16)
# plt.savefig('/Users/denistikhonov/Desktop/Study/IAD'
#             +'/Research_work/Spherical_Harmonics_parametrisation/doc/figs/tr_init.eps',
#             format='eps',
#             dpi=600,
#             bbox_inches='tight')
plt.show()

In [21]:
X = HankelMatrix(x_acc_clear,500)
X_init = HankelMatrix(x_acc,500)

In [22]:
tsne = TSNE(random_state=0,n_components=3)
X_tsne = tsne.fit_transform(X)

KeyboardInterrupt: 

In [ ]:
a = -120/180 * np.pi

T_X = np.array([[1,0,0],
                [0,np.cos(a),-np.sin(a)],
                [0,np.sin(a),np.cos(a)]])

T_Y = np.array([[np.cos(a),-np.sin(a),0],
                [np.sin(a),np.cos(a),0],
                [0,0,1]])

T_Z = np.array([[np.cos(a),0,np.sin(a)],
                [0,1,0],
                [-np.sin(a),0,np.cos(a)]])